In [1]:
import s3fs
from glob import glob
import xarray as xr

## Specify the S3 bucket where the NPS assets are

In [2]:
bucket = 's3://npwbanalres'

## Create a connection to the S3 bucket and list all of the assets 

In [3]:
s3 = s3fs.S3FileSystem(anon=False)

In [4]:
s3.glob(f'{bucket}/*.nc4')

['npwbanalres/v_1_5_1980_gridmet_historical_accumswe.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_accumswe_monthly.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_aet.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_deficit.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_deficit_monthly.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_pet.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_pet_monthly.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_rain.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_rain_monthly.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_runoff.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_runoff_monthly.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_soilwater.nc4',
 'npwbanalres/v_1_5_1980_gridmet_historical_soilwater_monthly.nc4',
 'npwbanalres/v_1_5_1981_gridmet_historical_accumswe.nc4',
 'npwbanalres/v_1_5_1981_gridmet_historical_accumswe_monthly.nc4',
 'npwbanalres/v_1_5_1981

In [28]:
years = list(set(int(v.split('_')[3].split('.')[0]) for v in s3.glob(f'{bucket}/*.nc4') if 'monthly' not in v))
print(f'Number of Years: {len(years)}')
print(f'Year Range: {min(years)} to {max(years)}')

Number of Years: 42
Year Range: 1980 to 2021


## List the variables for both daily and monthly assets

In [6]:
wb_variables = list(set(v.split('_')[-1].split('.')[0] for v in s3.glob(f'{bucket}/*.nc4') if 'monthly' not in v))
wb_variables

['soilwater', 'deficit', 'aet', 'accumswe', 'runoff', 'rain', 'pet']

In [7]:
wb_m_variables = list(set(f"{v.split('_')[-2]}_{v.split('_')[-1].split('.')[0]}" for v in s3.glob(f'{bucket}/*.nc4') if 'monthly' in v))
wb_m_variables

['aet_monthly',
 'rain_monthly',
 'accumswe_monthly',
 'runoff_monthly',
 'soilwater_monthly',
 'pet_monthly',
 'deficit_monthly']

## Connect to an NPS asset in S3

In [8]:
var = wb_m_variables[3]
var

'runoff_monthly'

In [9]:
urls = s3.glob(f'{bucket}/*{var}.nc4')
urls

### Read in a single asset

In [20]:
url = urls[0]
url

'npwbanalres/v_1_5_1980_gridmet_historical_runoff_monthly.nc4'

In [21]:
s3_file_obj = s3.open(url, mode='rb')

In [22]:
xr_ds = xr.open_dataset(s3_file_obj, chunks='auto', engine='h5netcdf')

In [23]:
xr_ds

<xarray.Dataset>
Dimensions:                  (time: 12, tbnds: 2, x: 4700, y: 3300)
Coordinates:
  * time                     (time) datetime64[ns] 1980-01-16 ... 1980-12-15
  * x                        (x) float32 -2.06e+06 -2.059e+06 ... 2.639e+06
  * y                        (y) float32 1.384e+06 1.383e+06 ... -1.915e+06
    lat                      (y, x) float32 dask.array<chunksize=(3300, 4700), meta=np.ndarray>
    lon                      (y, x) float32 dask.array<chunksize=(3300, 4700), meta=np.ndarray>
Dimensions without coordinates: tbnds
Data variables:
    time_bnds                (time, tbnds) datetime64[ns] dask.array<chunksize=(12, 2), meta=np.ndarray>
    timeunit                 (time) int16 dask.array<chunksize=(12,), meta=np.ndarray>
    lambert_conformal_conic  int16 ...
    runoff                   (time, y, x) float32 dask.array<chunksize=(6, 1980, 2822), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    NCO:          netCDF Operators version 5.0.4 (Homepage = http://nco.sf.ne...
    history:      Fri Jan 27 23:16:46 2023: ncatted -O -a units,time,o,c,days...

### Read in multiple assets

In [18]:
ds_all = []
for url in urls:
    #s3_file_obj = s3.open(url, mode='rb')
    ds_all.append(xr.open_dataset(s3.open(url, mode='rb'), chunks='auto', engine='h5netcdf'))

In [19]:
ds_ts = xr.concat(ds_all, dim='time')
ds_ts

<xarray.Dataset>
Dimensions:                  (time: 504, tbnds: 2, x: 4700, y: 3300)
Coordinates:
  * time                     (time) datetime64[ns] 1980-01-16 ... 2021-12-16
  * x                        (x) float32 -2.06e+06 -2.059e+06 ... 2.639e+06
  * y                        (y) float32 1.384e+06 1.383e+06 ... -1.915e+06
    lat                      (y, x) float32 51.78 51.79 51.79 ... 21.12 21.12
    lon                      (y, x) float32 -131.7 -131.7 ... -74.77 -74.76
Dimensions without coordinates: tbnds
Data variables:
    time_bnds                (time, tbnds) datetime64[ns] dask.array<chunksize=(12, 2), meta=np.ndarray>
    timeunit                 (time) int16 dask.array<chunksize=(12,), meta=np.ndarray>
    lambert_conformal_conic  (time) int16 -32767 -32767 -32767 ... -32767 -32767
    runoff                   (time, y, x) float32 dask.array<chunksize=(6, 1980, 2822), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    NCO:          netCDF Operators version 5.0.4 (Homepage = http://nco.sf.ne...
    history:      Fri Jan 27 23:16:46 2023: ncatted -O -a units,time,o,c,days...

---

In [ ]:
for v in wb_m_variables:
    print(v)
    urls = s3.glob(f'{bucket}/*{v}.nc4')
    #print(urls)
    ly_noly = [0,1,2,3,4,5,6,7,8,9,10]
    for y in ly_noly:
        #print(y)
        url = urls[y]
        print(url)
        try:
            s3_file_obj = s3.open(url, mode='rb')
            xr_ds = xr.open_dataset(s3_file_obj, chunks='auto', engine='h5netcdf')
            print(xr_ds.dims)
            print(xr_ds[v].attrs)
        except:
            print(f'FAILED: {url}')
    